In [60]:
#import tesseract for OCR and PILLOW to parse screenshots
from pytesseract import image_to_string
from PIL import Image
import pytesseract
from os import mkdir
import os
import pandas as pd

In [61]:
#Load categorizer and transformer from respective pickle files
from os import listdir
import sys
import pandas as pd
from sklearn.externals import joblib 
cat = joblib.load('categorizer.pkl')  
tran = joblib.load('transformer.pkl')

In [62]:
#path to the main directory of screenshots
mypath = "F:/screenshots/AMIT"

In [63]:
def categorizer(text_data):
    
    text = text_data.iloc[:,1]
    
    text_dtm = tran.transform(text.astype(str)) #transform to learned vocabulary
    
    predictions = (cat.predict(text_dtm)) #predict the categories
    text_data["predictions"] = predictions #add the predictions to the data frame
    call_corrector(text_data) #send the data frame to correction module

In [64]:
def call_corrector(dataframe):
    for index, row in dataframe.iterrows():
        if row["predictions"] == b"chat":
            correct_chat(row['path'])
        elif row['predictions'] == b'code':
            correct_code(row['path'])
        elif row['predictions'] == b'web':
            correct_web(row['path'])
        elif row['predictions'] == b'mail':
            correct_mail(row['path'])
        elif row['predictions'] == b'script':
            correct_script(row['path'])
        elif row['predictions'] == b'office':
            correct_office(row['path'])
        elif row['predictions'] == b'cloud':
            correct_cloud(row['path'])
        
        #os.remove(row['path'])

In [65]:
def correct_chat(text):
    print(text+"in chat")

def correct_web(text):
    print(text+" in web")

def correct_mail(text):
    print(text+" in mail")

def correct_script(text):
    print(text+" in script")

def correct_office(text):
    print(text+" in office")

def correct_cloud(text):
    print(text+" in cloud")

def correct_code(text):
    print(text+" in code")

In [66]:
def make_dir(path):
    mkdir(path)

In [67]:
def get_employee():
    employees_input=[] #list of input employee folders
    employees_output = [] #list of output employee folders
    
    for name in listdir(mypath):
        output_path = "F:/Output/"+name
        employees_output.append(output_path)
        if not os.path.exists(output_path):#if folder doesn't exists
                    make_dir(output_path)
        employees_input.append(mypath+"/"+name)
        
    date_folders(employees_input, employees_output)
 


In [68]:

def date_folders(employees_input, employees_output):
        
    dates_input = [] #list of input employee->date folder
    dates_output = [] #list of output employee->date folder
    
    #get date folder names inside every employee's folder
    for a, b in zip(employees_input, employees_output):
        for date in listdir(a):
            output_path = b+"/"+date
            dates_output.append(output_path)
            if not os.path.exists(output_path): #if folder doesn't exists
                    make_dir(output_path)
            dates_input.append(a+"/"+date)
            
    screenshots_to_text(dates_input, dates_output)


In [69]:
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe'
def screenshots_to_text(dates_input, date_output):
    size = 7016, 4961 #size for 300 dpi image
    
    screenshots = [] #list of input employee->date->screenshots
    screenshot_output_names = [] #list of input employee->date->screenshot names
    
    for a, b  in zip(dates_input, date_output):
        for screenshot in listdir(a):
            screenshot_output_names.append(b+"/"+screenshot)
            screenshots.append(a+"/"+screenshot)
            
        path = []
        text = []
        #implement pytesseract's OCR here
        for image, image_name in zip(screenshots, screenshot_output_names):
            print(f"image = {image}") 
            print(f"image_name = {image_name}")
            if not os.path.exists(image_name+".txt"): #if file doesn't exist
                im = Image.open(image)
                im = im.resize(size, Image.ANTIALIAS) #resize image to 300dpi for better results
                
          #create a list (path) for image_name beacuse it contains the path to those OCR outputs
                file_path = image_name+".txt"
                path.append(file_path) 
               
                #open a txt file for OCR results 
                f = open(file_path,"w+")  
                
                #create a list(text) for image_to_string outputs
                output = image_to_string(im, lang="eng+fra")
                text.append(output)
                            
        #add the OCR results to the screenshot.txt
                f.write(output)
            
            
        #close the output file
                f.close()
                im.close()
        if len(path) > 0:
            data = {"path": path, "text":text}  #create a dictionary with path and text lists
            text_data = pd.DataFrame(data)      #create a pandas dataframe with the above dictionary
            categorizer(text_data)              #pass this dataframe to categorizer
           
           

In [70]:
#get_employee()

In [ ]:
employees_input = ["F:/screenshots/DILIP"]
employees_output = ["F:/Output/DILIP"]
date_folders(employees_input, employees_output)